# Text Generation

Exploring raw text generation using llama-jax with and without KV caching.

# Setup

In [11]:
from contextlib import contextmanager
from sys import stdout
from time import time_ns, perf_counter_ns as timer

import jax
from jax import Array
from jax import numpy as jnp
from jax import random

import llama_jax as ll

In [2]:
print(f"Available devices: {jax.devices()}")

Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB

Available devices: [METAL(id=0)]


W0000 00:00:1740771462.328766 7635710 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1740771462.344881 7635710 service.cc:145] XLA service 0x1179bb7a0 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740771462.344891 7635710 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1740771462.346100 7635710 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1740771462.346108 7635710 mps_client.cc:384] XLA backend will use up to 51539214336 bytes on device 0 for SimpleAllocator.


In [3]:
@contextmanager
def tps_report(n_tokens: int):
    start_time = timer()

    yield
    
    stdout.write("\n\n")
    
    duration = (timer() - start_time) / 1000000000
    tps = n_tokens / duration
    
    print(f"Generated {n_tokens} tokens in {duration:0.1f} s ({tps:0.1f} tps)")

# Configure

In [4]:
# Parameters
max_tokens = 20
seed = time_ns()

In [5]:
# Configure model
key = random.key(seed)
config = ll.checkpoint.load_config("Llama3.2-3B")
params = ll.checkpoint.load_parameters(config)
tokenizer = ll.checkpoint.load_tokenizer(config)
model = ll.model.create(config, params)

# Prompt

In [6]:
prompt = "I like traveling by train because"

# Without KV Cache

In [9]:
token_ids, position_mask = tokenizer.encode(prompt)
token_ids

Array([[128000,     40,   1093,  21646,    555,   5542,   1606]], dtype=int32)

In [ ]:
def transform(config, model, x, position_mask):


In [10]:
with tps_report(max_tokens):    
    
    # Print input tokens
    stdout.write(prompt)
    
    # Process all tokens on first pass
    x = token_ids
    
    for _ in range(max_tokens):
        # Transform tokens into logits
        logits = ll.model.forward(config, model, x, position_mask)
        
        # Sample next token
        key, subkey = random.split(key)
        next_token_id = ll.model.next_token(logits, key=subkey)
        
        # Print next token
        stdout.write(tokenizer.decode(next_token_id)[0])
    
        # Process all tokens on next pass
        x = jnp.concat([x, next_token_id], axis=-1)

I like traveling by train because it’s the most convenient way to get around. I don’t have to worry about driving or parking

Generated 20 tokens in 28.4 s (0.7 tps)


# With KV Cache

In [ ]:
token_ids, position_mask = tokenizer.encode(prompt)
token_ids, position_mask

In [ ]:
with tps_report(max_tokens):    
    
    # Print input tokens
    stdout.write(prompt)
    
    # Initialize cache
    kv_cache = ll.kv_cache.create(config)
    
    # Process all tokens on first pass
    x = token_ids
    
    for _ in range(max_tokens):
        # Transform tokens into logits
        logits, kv_cache = ll.model.forward(config, model, x, position_mask, kv_cache=kv_cache)
        
        # Sample next token
        key, subkey = random.split(key)
        next_token_id = ll.model.next_token(logits, key=subkey)
        
        # Print next token
        stdout.write(tokenizer.decode(next_token_id)[0])
    
        # Process generated token on next pass
        x = next_token_id
        position_mask = ll.model.increment_position_mask(position_mask)        